In [167]:
import tensorflow as tf
print(tf.__version__)

In [168]:
from deepexplain.tf.v2_x import DeepExplain
from EEGModels import EEGNet
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import os
import scipy
import numpy as np
from sklearn.metrics import balanced_accuracy_score


In [169]:
folder_path = '../capped_data/data/walk_instruction_raw/'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.mat')]
file_list.sort()

file_dict = {index: file for index, file in enumerate(file_list)}

X = []
Y = []

for i, file in enumerate(file_list):
    x = scipy.io.loadmat(file)
    x = x['data']

    x = x[:,:,:500]

    print(x.shape)
    
    first_letter = file.split('/')[-1][0]

    # create output vector. If file begins with 'P', then y = [1,0], else y = [0,1], having the same dimension as x
    if first_letter == 'P':
        y = np.zeros((x.shape[0],2))
        y[:,0] = 1
    else:
        y = np.zeros((x.shape[0],2))
        y[:,1] = 1

    X.append(x)
    Y.append(y)

(56, 14, 500)
(37, 14, 500)
(49, 14, 500)
(56, 14, 500)
(58, 14, 500)
(57, 14, 500)
(60, 14, 500)
(60, 14, 500)
(67, 14, 500)
(60, 14, 500)
(80, 14, 500)
(79, 14, 500)
(66, 14, 500)
(84, 14, 500)


In [170]:
kernels, chans, samples = 1, 14, 500

In [171]:
index_1 = 2
index_2 = 8

X_train = [X[i] for i in range(len(X)) if i != index_1 and i != index_2]
Y_train = [Y[i] for i in range(len(Y)) if i != index_1 and i != index_2]

# build test set from the files with index_1 and index_2
X_test = [X[i] for i in range(len(X)) if i == index_1 or i == index_2]
Y_test = [Y[i] for i in range(len(Y)) if i == index_1 or i == index_2]

X_train = np.concatenate(X_train)
Y_train = np.concatenate(Y_train)
X_test = np.concatenate(X_test)
Y_test = np.concatenate(Y_test)

# convert data to NHWC (trials, channels, samples, kernels) format.
X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)


In [172]:
model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

# count number of parameters in the model
numParams    = model.count_params()     

class_weights = {0:1, 1:1}

fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 15, verbose = 2, 
                        class_weight=class_weights)

Train on 753 samples
Epoch 1/15
753/753 - 5s - loss: 0.6818 - acc: 0.5697 - 5s/epoch - 6ms/sample
Epoch 2/15
753/753 - 4s - loss: 0.6477 - acc: 0.6308 - 4s/epoch - 5ms/sample
Epoch 3/15
753/753 - 4s - loss: 0.6377 - acc: 0.6401 - 4s/epoch - 6ms/sample
Epoch 4/15
753/753 - 4s - loss: 0.6288 - acc: 0.6547 - 4s/epoch - 5ms/sample
Epoch 5/15
753/753 - 4s - loss: 0.6006 - acc: 0.6972 - 4s/epoch - 5ms/sample
Epoch 6/15
753/753 - 4s - loss: 0.6048 - acc: 0.6999 - 4s/epoch - 5ms/sample
Epoch 7/15
753/753 - 4s - loss: 0.5977 - acc: 0.6999 - 4s/epoch - 5ms/sample
Epoch 8/15
753/753 - 3s - loss: 0.5982 - acc: 0.6932 - 3s/epoch - 5ms/sample
Epoch 9/15
753/753 - 4s - loss: 0.5806 - acc: 0.7092 - 4s/epoch - 5ms/sample
Epoch 10/15
753/753 - 4s - loss: 0.5911 - acc: 0.6919 - 4s/epoch - 5ms/sample
Epoch 11/15
753/753 - 4s - loss: 0.5816 - acc: 0.6985 - 4s/epoch - 5ms/sample
Epoch 12/15
753/753 - 3s - loss: 0.5719 - acc: 0.6999 - 3s/epoch - 5ms/sample
Epoch 13/15
753/753 - 3s - loss: 0.5709 - acc: 0.713

In [173]:
print(model.summary())

Model: "model_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 14, 500, 1)]      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 500, 8)        256       
                                                                 
 batch_normalization_30 (Bat  (None, 14, 500, 8)       32        
 chNormalization)                                                
                                                                 
 depthwise_conv2d_10 (Depthw  (None, 1, 500, 16)       224       
 iseConv2D)                                                      
                                                                 
 batch_normalization_31 (Bat  (None, 1, 500, 16)       64        
 chNormalization)                                                
                                                          

In [175]:
# np.float32(Y_test)
# print(X_test.dtype, X_test.shape)
# print(Y_test.dtype, Y_test.shape)
X = tf.placeholder("float", [None, chans, samples, kernels])
Y = tf.placeholder("float", [None, 2])
print(X.dtype, X.shape)
print(Y.dtype, Y.shape)

with DeepExplain(session = tf.compat.v1.keras.backend.get_session()) as de:
	input_tensor = model.input
	fModel         = Model(inputs = input_tensor, outputs = model.layers[-2].output)    
	target_tensor  = fModel(input_tensor)    

	# can use epsilon-LRP as well if you like.
	attributions   = de.explain('deeplift', target_tensor, input_tensor, X_test, ys=Y_test)
	# attributions = de.explain('elrp', target_tensor * Y_test, input_tensor, X_test)


float32 (116, 14, 500, 1)
float64 (116, 2)
<dtype: 'float32'> (?, 14, 500, 1)


InvalidArgumentError: Graph execution error:

Detected at node 'input_9' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\HTPDIR-Laptop1\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Users\HTPDIR-Laptop1\AppData\Local\Programs\P